[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/farahbakhsh3/X1/blob/master/Ferdosi2.ipynb)

In [4]:
# !wget https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt

--2018-10-17 05:23:46--  https://www.dropbox.com/s/laax0sxqzzlddtq/ferdosi.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:601a:1::a27d:701
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/laax0sxqzzlddtq/ferdosi.txt [following]
--2018-10-17 05:23:46--  https://www.dropbox.com/s/raw/laax0sxqzzlddtq/ferdosi.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc449e2a88aadf43dd65482ffb28.dl.dropboxusercontent.com/cd/0/inline/ATM_-9_waqbmI49BK1cvUZp2Z16PIG2FJE1f73_Z2v7j1VDFaTP3vVGaiVrX2vuNddaQ0Is4YUqZeLZYYY-WLkKZh3WuuIu2OjSj_RmdG2YkrFdBJXfcGOFLm3Vx95BetF5bN0nuu0i9IbJzePSriRsR_MTWreYIm7PcAknwplr2C2bC7WvgSI-dO71MSPTqX0s/file [following]
--2018-10-17 05:23:46--  https://uc449e2a88aadf43dd65482ffb28.dl.dropboxusercontent.com/cd/0/inline/ATM_-9_waqbmI49BK1cvUZp2Z16PIG2FJE1f73_Z2v7j1VDFaTP3vV

In [0]:
import random
 
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout
from keras.layers import LSTM, GRU, CuDNNGRU, CuDNNLSTM 
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences


In [6]:
path = './ferdosi.txt'
with open(path, encoding='utf-8') as f:
    texts = f.read()
print('corpus length:', len(texts))

texts = texts.replace('\n', ' ')
texts = texts.split(' ')

corpus length: 2545108


In [7]:
max_length = 4
step = 2
sentences = []
next_words = []
for i in range(0, len(texts) - max_length, step):
    sentences.append(texts[i: i + max_length])
    next_words.append(texts[i + max_length])
print('nb sequences:', len(sentences))

nb sequences: 280900


In [8]:
print(sentences[0])
print(next_words[0])

['به', 'نام', 'خداوند', 'جان']
و


In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 19256 unique tokens.


In [0]:
len_of_words = len(word_index)
embeding_dim = 32

word2idx = dict((c, i) for i, c in enumerate(word_index))
idx2word = dict((i, c) for i, c in enumerate(word_index))

In [11]:
x = pad_sequences(sequences, maxlen=max_length)
# x = np.array(sequences)
y = np.zeros((len(sequences)), dtype=int)
for i, sentence in enumerate(sentences):
    try:
        y[i] = int(word2idx[next_words[i]])
    except:
        pass

print('Shape of data tensor:', x.shape)
print('Shape of label tensor:', y.shape)

Shape of data tensor: (280900, 4)
Shape of label tensor: (280900,)


In [12]:
print(1, [idx2word[idx-1] for idx in x[0]])
print(2, idx2word[y[0]])

print(3, x[0])
print(5, word2idx[next_words[0]])
print(6, idx2word[word2idx[next_words[0]]])

1 ['به', 'نام', 'خداوند', 'جان']
2 و
3 [  2  83 354 115]
5 0
6 و


In [0]:
def on_epoch_end(epoch, _):
    if epoch % 5 != 0:
        return
    
    start_index = random.randint(0, len(texts) - max_length - 1)
    for diversity in [1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = ' '.join(texts[start_index: start_index + max_length])
        print(sentence)
        generated = generated.join(sentence)
        print('----- Generating with seed: "' + sentence + '"')
        for i in range(160):
            x_pred = np.zeros((1, max_length), dtype=int)
            for t, word in enumerate(sentence.split(' ')):
                x_pred[0, t] = word2idx[word]

            preds = model.predict(x_pred, verbose=0)[0]
            pred_id = sample(preds, diversity)
            next_word = idx2word[pred_id]

            generated = generated + ' ' + next_word
            sentence = ' '.join(sentence.split(' ')[1:])
            sentence = sentence + ' ' + next_word

        print(generated.replace('**', '\n'))

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
# model = models.Sequential()
# model.add(layers.Embedding(len(word_index), 100))
# model.add(layers.LSTM(128))
# model.add(layers.Dense(1, activation='sigmoid'))

# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


model = Sequential()
model.add(Embedding(len_of_words, embeding_dim))
model.add(CuDNNGRU(256))
# model.add(GRU(256))
model.add(Dense(len_of_words, activation='softmax'))
 
model.compile(optimizer=RMSprop(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
 
print(model.summary())
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(x=x, y=y, epochs=60, batch_size=128, callbacks=[print_callback])


In [16]:
history = model.fit(x=x, y=y, epochs=60, batch_size=128, callbacks=[print_callback])

Epoch 1/60
280900/280900 [==============================] - 43s 152us/step - loss: 5.5915 - acc: 0.2495
----- diversity: 1.0
گرگ و میش مرا
----- Generating with seed: "گرگ و میش مرا"


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


گرگ و میش مرا هم آورد گشته نباید تو روزگار افراسیاب انجمن و سام بدی روز نیست جام ما گوید نیاید سر او گر کارزار هزار خیره نیست بدو از داد فریدون چشم شب گفت تو او جنگ زن و دیو و گرم خردمند خسرو یک چشم کیان بزد همه گفت سپهبد پر پاک داد پر پر شاد بداندیش دو گناه آن بوم با کوه تخت کردم که اندیشه مرا سرافراز پدر تا پیش را سپاه کسی ز من بشد را چه آید من آباد به چیز ز خود گشتاسپ به پدر و به همان نگه اندازه و ابا زمین بسی را خاک خاک باید چیست فر بود چه گیتی بدو از خاک چون و چه آید پاک نگه مگر آوریم آنک انجمن و کشته نو نامور ز یک سیاوش آمد زال چون به بی کاووس بوم هم فرزند رستم و آواز کنی مگر دستان دو تا نهان باز به چین مرا گیتی اگر کیان سوی گر آرزو زن گرم نباید خسرو هر گفت
Epoch 2/60
280900/280900 [==============================] - 43s 152us/step - loss: 5.6046 - acc: 0.2509
Epoch 3/60
280900/280900 [==============================] - 43s 152us/step - loss: 5.6273 - acc: 0.2495
Epoch 4/60
280900/280900 [==============================] - 43s 152us/step - loss: 5.6428 - acc: 0.2492
Epoch 5/60
28090

In [0]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import CuDNNLSTM


# load ascii text and covert to lowercase
filename = "ferdosi.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 5):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Total Characters:  2545108
Total Vocab:  37
Total Patterns:  509002
Epoch 1/20
  4480/509002 [..............................] - ETA: 17:05 - loss: 3.1169

In [51]:
# load the network weights
# filename = "weights-improvement-19-1.9435.hdf5"
# model.load_weights(filename)
# model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
xx = ''
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    xx = xx + result
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print ("\nDone.")

Seed:
" ول جای
نپرید بر آسمان بر همای
ز دیوان جنگی ده و دو هزار
به شب پاسبانند بر چاهسار
چو پولاد غندی سپهدا "

Done.


In [53]:
print(xx)

ن بوی
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به مر بو بر نید و بر بود و داه
به